In [1]:
import os
import re
import numpy as np
import pandas as pd

In [ ]:
### 1. 删除不符合要求的数据点
categories = ["train", "val", "test"]
for category in categories:
    file_path = "/lustre1/g/geog_geors/skguan/dataset_23(season)_num12.csv"
    df = pd.read_csv(file_path)
    df = pd.DataFrame(df[f'{category}'].dropna(), columns=[f'{category}'])
    pattern = "(?P<site>.+) (?P<year>\\d+)-(?P<month>\\d+)-(?P<day>\\d+)"
    with open("logs/training_20250625_vit(Season_2Sat)_2164022.out", 'r') as f:
        lines = f.readlines()
        for line in lines:
            if "00:00:00" in line:
                sites = np.array(list(map(lambda x: x.split("/")[-2], df[f'{category}'])))
                dates = np.array(list(map(lambda x: re.match(".*_\\d{6}_(\\d{8})*", x.split("/")[-1]).group(1),
                                        df[f'{category}'])))
                regex = re.match(pattern, line)
                site = regex.group("site")
                date = regex.group("year") + regex.group("month") + regex.group("day")
                idx = np.argwhere((sites == site) & (dates == date))
                if len(idx) == 0:
                    continue
                index = int(idx.squeeze())
                index_names = df[df[f'{category}'] == df[f'{category}'].iloc[index]].index
                df.drop(index_names, inplace=True)
    df.to_csv(os.path.splitext(file_name)[0] + f"_{category}.csv")

In [2]:
### 2. 删除重复的数据点
file_path = "/lustre1/g/geog_geors/skguan/dataset_23(season)_num6.csv"
df = pd.read_csv(file_path)
df_stack = df[df.columns[1:]].stack().reset_index(drop=True)
df_sorted = df_stack.sort_values(key=lambda x: x.map(
    lambda y: re.match(".+_(\\d{6})_.+", os.path.basename(y)).group(1)
))
file_num = len(df_sorted)
tiles = set(map(
    lambda x: re.match("\\w+_(\\d{6})_", os.path.split(x)[-1]).group(1),
    df_sorted.values
))
dataset = {
    "train": [],
    "val": [],
    "test": []
}
train_num = int(file_num * .8)
val_num = train_num + int(file_num * .1)
num = 0
for tile in tiles:
    temp_files = filter(
        lambda x: re.match(f"LC09.*_{tile}_2023.*.tif", os.path.basename(x)),
        df_sorted.values
    )
    temp_files = list(temp_files)
    num += len(temp_files)
    train_ratio = (num - train_num) / file_num
    val_ratio = (num - val_num) / file_num
    if train_ratio < 0:
        dataset["train"].extend(temp_files)
    elif val_ratio < 0:
        dataset["val"].extend(temp_files)
    else:
        dataset["test"].extend(temp_files)
df = pd.DataFrame(
    dict([(k, pd.Series(v)) for k, v in dataset.items()])
)
df.to_csv(file_path)